In [7]:
'''
#  ELM MODULE  
#================================================================================
#  Due to limited GPU RAM - input size should be smaller than 5000 on Kaggle 
#  On other platforms it depends on the available resources. 
#  For larger inputs size is better to use the latest cell (implementing a
#  typical multilayer perceptron in Keras)
# Copyright Radu & Ioana DOGARU - radu_d@ieee.org 
# More details in paper 
# R. Dogaru and Ioana Dogaru, "BCONV-ELM: Binary Weights Convolutional 
# Neural Network Simulator based on Keras/Tensorflow for Low Complexity 
# Implementations", in Proceedings ISEEE 2019, in Press. 
# Please cite the above paper if you find this code useful 
# 
#--------------------------------------------------------------------------
'''
import keras.backend as K
import tensorflow as tf
import scipy.linalg as sclin
import numpy as np
import time as ti
import scipy.io as sio
# ------------------------  ELM module parameters -------------
dataset='f-mnist' # it can be mnist, cifar10, f-mnist, other loads a local file
ifother='...' # depends on your available files (they should be in .mat format)
nr_neuroni=6000 # Proposed number of neurons on the hidden layer 
#C=0.100000 # Regularization coefficient C  (small value / useful for 0 neurons)
C=10 # Useful in the case of a hidden layer with N>>0 neurons 
tip=3 # Nonlinearity of the hidden layer (-1 means linear layer)
if nr_neuroni==0:
    tip=-1   # 
nb_in=2;  # 0 = float; x - represents weights on a finite x number of bits 
nb_out=8; # same as above but for the output layer
first_samples=40000; # 0 - all samples ; [value] - first [value] samples

# =============  ELM basic functions 
def hidden(x_,inw_,tip):
# Hidden layer definit ca "flow" Keras (argumentele sunt "variables")
      hin_=K.dot(inw_,x_)
      #----------  HIDDEN LAYER --------- 
      if tip==-1:  # liniar (pentru Adaline only)
        h_=hin_
      elif tip==0: # tanh
        h_=K.tanh(hin_)
      elif tip==1:  # linsat 
        h_=K.abs(1+hin_)-K.abs(1-hin_)
        # de verificat daca 1+ merge ... 
      elif tip==2: # ReLU
        h_=K.relu(hin_)
      elif tip==3: 
            h_=K.abs(hin_)
      elif tip==4:
            h_=K.sqrt(K.square(hin_)+1)
      #------------------------------------ 
      return h_

# implements the ELM training procedure with weight quantization       
def elmTrain_fix( X, Y, h_Neurons, C , tip, ni):
# Training phase - emulated fixed point precision (ni bit quantization)
# X - Samples (feature vectors) Y - Labels
# ni - number of bits to quantize the inW weights 
      Ntr = np.size(X,1)
      in_Neurons = np.size(X,0)
      classes = np.max(Y)
      # transforms label into binary columns  
      targets = np.zeros( (classes, Ntr), dtype='int8' )
      for i in range(0,Ntr):
          targets[Y[i]-1, i ] = 1
      targets = targets * 2 - 1
      
      #   Generare inW 
      #   Generate inW layer 
      #   Takes care if h_Neurons==0 
      if h_Neurons==0:
          inW=np.eye(in_Neurons)
          h_Neurons=in_Neurons
    
      else: 
          rnd = np.random.RandomState()
          inW=-1+2*rnd.rand(h_Neurons, in_Neurons).astype('float32')
          #inW=rnd.randn(nHiddenNeurons, nInputNeurons).astype('float32')
          if ni>0:
            Qi=-1+pow(2,ni-1) 
            inW=np.round(inW*Qi)
      
      #  Compute hidden layer 
      iw_=K.variable(inW)
      x_=K.variable(X)
      h_=hidden(x_,iw_,tip)  
      #------------------------------------      
      # Moore - Penrose computation of output weights (outW) layer 
      ta_=K.variable(targets)
      print('KERAS ACTIVE')
      if h_Neurons<Ntr:
          print('LLL - Less neurons than training samples')
          outw_=tf.matrix_solve(K.eye(h_Neurons)/C+K.dot(h_,K.transpose(h_)),K.dot(h_,K.transpose(ta_)))  
      else:
          print('MMM - More neurons than training samples')
          outw_=K.dot(h_,tf.matrix_solve(K.eye(Ntr)/C+K.dot(K.transpose(h_),h_),K.transpose(ta_)))
      outW=K.eval(outw_)   
      K.clear_session()     
      return inW, outW 
      

def elmPredict_optim( X, inW, outW, tip):
# implements the ELM predictor given the model as arguments 
# model is simply given by inW, outW and tip 
# returns a score matrix (winner class has the maximal score)
      x_=K.variable(X)
      iw_=K.variable(inW)
      ow_=K.variable(outW)
      h_=hidden(x_,iw_,tip) 
      mul1=K.dot(K.transpose(h_),ow_)
      sc_=K.transpose(mul1)
      score = K.eval(sc_)
      K.clear_session() 
      return score 
        
def read_mat_data(nume):
    # reads data saved in the LIBSVM  .mat format (Samples Labels each from name_train & name_test)
    # 
    db=sio.loadmat(nume+'_train.mat')
    Samples=db['Samples'].astype('float32')
    x_train=Samples.T
    Labels=db['Labels'].astype('float32')
    y_train=-1+Labels.T[:,0]
    db=sio.loadmat(nume+'_test.mat')
    Samples=db['Samples'].astype('float32')
    x_test=Samples.T
    Labels=-1+db['Labels'].astype('float32')
    y_test=Labels.T[:,0]
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    return (x_train,x_test,y_train,y_test)

#===============  TRAIN DATASET LOADING ==========================================
from keras.datasets import mnist, cifar10, fashion_mnist

if dataset=='mnist':
    (x_train, y_train), (x_test, y_test) = mnist.load_data() # incarca date nescalate 
elif  dataset=='cifar10': 
    (x_train, y_train), (x_test, y_test) = cifar10.load_data() # incarca date nescalate 
elif dataset=='f-mnist':
    (x_train, y_train), (x_test, y_test) =  fashion_mnist.load_data()
elif dataset=='other': #load some local file of your choice (edit the name in the next lines)
    nume='../input'+ifother
    (x_train,x_test,y_train,y_test)=read_mat_data(nume)
    
if (np.ndim(x_train)==3):   # E.g.  MNIST or F-MNIST  
    x_train=np.reshape(x_train, [np.shape(x_train)[0],np.shape(x_train)[1],np.shape(x_train)[2], 1]) 
    x_test=np.reshape(x_test, [np.shape(x_test)[0],np.shape(x_test)[1],np.shape(x_test)[2], 1] ) 
# place a  1 in the end to keep it compatible with kernel in conv2d 
# scaling in ([0,1])
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /=255 
#=============================================================================
# converts x_train, y_train into Samples Labels 
intrain=K.variable(x_train)
Samples_=K.batch_flatten(intrain)  # aici se aplica direct datele de intrare 
Samples=(K.eval(Samples_)).T 
Labels=(y_train.T+1).astype('int8')
if (np.ndim(Labels)<2): 
    Labels=np.reshape(Labels,[1,np.shape(Labels)[0]])
clase=np.max(Labels)
#================= TRAIN ELM =====================================================
t1 = ti.time()
inW, outW = elmTrain_fix(Samples, np.transpose(Labels), nr_neuroni, C, tip, nb_in)
trun = ti.time()-t1
print(" training time: %f seconds" %trun)
# ==============  Quantify the output layer ======================================
Qout=-1+pow(2,nb_out-1)
if nb_out>0:
     O=np.max(np.abs(outW))
     outW=np.round(outW*(1/O)*Qout)
#================= TEST (VALIDATION) DATASET LOADING 
intest=K.variable(x_test)
Samples_=K.batch_flatten(intest)  # aici se aplica direct datele de intrare 
Samples=(K.eval(Samples_)).T
Labels=(y_test.T+1).astype('int8')
if (np.ndim(Labels)<2):
    Labels=np.reshape(Labels,[1,np.shape(Labels)[0]])  # acopera cazul MNIST 

n=Samples.shape[0]
N=Samples.shape[1]
#====================== VALIDATION PHASE (+ Accuracy evaluation) =================
t1 = ti.time()
scores = elmPredict_optim(Samples, inW, outW, tip)
trun = ti.time()-t1
print( " prediction time: %f seconds" %trun)

# CONFUSION MATRIX computation ==================================
Conf=np.zeros((clase,clase),dtype='int16')
for i in range(N):
    # gasire pozitie clasa prezisa 
    ix=np.where(scores[:,i]==np.max(scores[:,i]))
    ixx=np.array(ix)
    pred=int(ixx[0,0])
    actual=Labels[0,i]-1
    Conf[actual,pred]+=1
accuracy=100.0*np.sum(np.diag(Conf))/np.sum(np.sum(Conf))
print("Confusion matrix is: ")
print(Conf)
print("Accuracy is: %f" %accuracy)
print( "Number of hidden neurons: %d" %nr_neuroni)
print( "Hidden nonlinearity (0=sigmoid; 1=linsat; 2=Relu; 3 - ABS; 4- multiquadric): %d" %tip)
K.clear_session() 
#====================================================================================   

KERAS ACTIVE
LLL - Less neurons than training samples
 training time: 7.904217 seconds
 prediction time: 0.869411 seconds
Confusion matrix is: 
[[831   1  20  28   5   5  97   0  13   0]
 [  3 968   1  17   3   1   5   0   2   0]
 [ 14   0 801  12 102   0  68   0   3   0]
 [ 21   3  15 895  30   0  30   0   5   1]
 [  1   0  95  34 804   2  60   0   4   0]
 [  0   0   0   1   0 959   0  28   1  11]
 [139   1  90  28  73   3 648   0  18   0]
 [  0   0   0   0   0  13   0 954   0  33]
 [  1   0   4   4   3   3   8   4 973   0]
 [  0   0   0   0   0   5   1  27   0 967]]
Accuracy is: 88.000000
Number of hidden neurons: 6000
Hidden nonlinearity (0=sigmoid; 1=linsat; 2=Relu; 3 - ABS; 4- multiquadric): 3
